#### Students Name and ID:
- XXXXXXX XXXXXXX, 00000000
- XXXXXXX XXXXXXX, 00000000

#### Step 1: Generating features and labels

In [0]:
from pyspark.sql import functions as f
from pyspark.sql.dataframe import DataFrame

def generate_features(input_df: DataFrame, start_date: str, end_date:str) -> DataFrame:
    """
    Generate features from the input DataFrame.
    
    :param input_df: Input DataFrame containing order information.
    :param start_date: Start date for filtering the input DataFrame.
    :param end_date: End date for filtering the input DataFrame.

    :return: DataFrame with generated features.
    """
    features_df = (
        ...
    )
    
    return features_df

In [0]:
def generate_labels(input_df: DataFrame, y_reference_month: str) -> DataFrame:
    """
    Generate labels for the input DataFrame.
    
    :param input_df: Input DataFrame containing order information.
    :param y_reference_month: reference month for filtering the input DataFrame.
    
    :return: DataFrame with generated labels.
    """
    labels_df = (
        ...
    )
    return labels_df

In [0]:
def generate_features_and_labels(input_df: DataFrame) -> DataFrame:
    features_df = input_df.transform(lambda df: generate_features(df, ..., ...))
    labels_df = input_df.transform(lambda df: generate_labels(df, '2021-06-01'))
    output_df = labels_df.join(features_df, ['account_id'], 'inner')
    
    return output_df

> **Features documentation**

- label (y): ...
- feature 1: ...
- feature 2: ...
...

#### Step 2: The ML Pipeline

In [0]:
from pyspark.ml import Pipeline, PipelineModel

pipe = Pipeline()
pipe.setStages(
    [
        ...
    ]
)

Out[10]: Pipeline_425f36845461

> **Why did you choose each step of your ML Pipeline?**

- Step "preprocessingMethod": ...
- Step "machinelearningModel": ...  
...

#### Step 3: Tuning the hyperparameters of your model

In [0]:
# Define the evaluator
...

# Define the training loop, logging metrics to ml flow (refer to lab 12)
...

>**Which metrics did you choose for evaluation? Justify.**

...

>**Present the results of your hyperparameter tuning in the next cell. Is there any parameter more correlated to the evaluation metrics? What is your hypothesis about this?**
*(tip: go for 2 parameter in your grid, it makes it easy to show the results and you can present them in a heat map)*

...

In [0]:
# Read experiment data from mlflow and present the result

#### Step 4: Generate the targetting with your model and analyze the results of `JUNE`

In [0]:
def test_results_applying_target(test_month: str = '2021-06-01', target: DataFrame = None):

    orders_sample = spark.read.parquet(f"s3://dev-ifood-kairos/misc/bda/ord_sample")
    test_sample = spark.read.parquet(f"s3://dev-ifood-kairos/misc/bda/mkt_test_assignment_sample")
    
    orders_by_account = (
      orders_sample
      .filter( f.date_trunc('month', f.col('reference_date'))=='2021-06-01')
      .groupBy('account_id', 'reference_date')
      .agg(
          f.countDistinct('order_id').alias('orders'),
          f.sum('discount_vouchers_used').alias('vouchers_used'),
          f.sum('subsidy').alias('subsidy'),
          f.avg('order_total').alias('aov')
      )
    )

    if target:
      test_sample = ( test_sample.join(target, ['account_id'], 'inner') )

    (
      test_sample
      .filter(f.col('reference_month')==test_month)
      .join(orders_by_account,  ['account_id', 'reference_date'], 'left')
      .groupBy('test_assignment')
      .agg(
          f.count('account_id').alias('test_size'),
          f.round((f.sum( f.when(f.col('orders') > 0, 1).otherwise(0) )), 3).alias('conversions'),
          f.round((f.sum( f.when(f.col('orders') > 0, 1).otherwise(0) ) / f.count('account_id') ), 3).alias('conversion_rate'),
          f.round(f.sum('subsidy'), 0).alias('sub_total'),
          (f.sum( f.when(f.col('vouchers_used') > 0, 1).otherwise(0) ) ).alias('vouchers_used'),
          f.avg('aov').alias('aov')
      )
      .display()
      
    )

In [0]:
target_customers =(
  ...
)

In [0]:
test_results_applying_target('2021-06-01')

test_assignment,test_size,conversions,conversion_rate,sub_total,vouchers_used,aov
control,147962,10899,0.074,0.0,0,2330.0940879897103
test,164460,12093,0.074,339914.0,882,2297.9830366740953


> **Analyze the results. What are your conclusions? Remember to compare the results to the baseline targetting.**

...

#### Step 4: Generate the targetting with your model and analyze the results of `JULY`. 
*(the data of july will be made available 24 hours before the submission deadline)*

In [0]:
target_customers =(
  ...
)

In [0]:
test_results_applying_target('2021-06-01')

> **What is the resulting iMUB and CPI of your targetting?**  
iC = ...  
CPI = ...

> **Analyze the results. What are your conclusions?**

...